In [ ]:
# --- Environment Setup ---
import sys
print("Python executable:", sys.executable)

# Install required packages into THIS environment
!{sys.executable} -m pip install numpy scipy matplotlib pandas

# Hybrid Digital Twin - Pharma Style

This notebook demonstrates a hybrid digital twin framework combining mechanistic models with machine learning corrections for pharmaceutical process simulation.

# Governing Equations and Modeling Assumptions

## Process Description
This project models a non-isothermal batch reactor carrying out consecutive first-order liquid-phase reactions:

A → B → C

The objective is to maximize the concentration of intermediate B by controlling the reactor temperature profile and batch time.

This formulation is representative of batch pharmaceutical and specialty chemical manufacturing processes where temperature history strongly affects yield and impurity formation.

The mechanistic model follows standard formulations commonly presented in process dynamics and control literature.

## Assumptions
* Perfect mixing in the reactor
* Constant reactor volume
* Constant density and heat capacity
* Liquid-phase reactions
* No inflow or outflow during the batch
* Lumped heat transfer model
* Jacket temperature is treated as a known input

## Reaction Kinetics
The reactions are first-order with Arrhenius temperature dependence:

$$k_1(T) = k_{0,1} \exp\left(\frac{-E_1}{RT}\right)$$

$$k_2(T) = k_{0,2} \exp\left(\frac{-E_2}{RT}\right)$$

**Where:**
- $k_1, k_2$ = reaction rate constants
- $k_{0,1}, k_{0,2}$ = pre-exponential factors
- $E_1, E_2$ = activation energies
- $R$ = universal gas constant
- $T$ = reactor temperature

## Mass Balances

### Component A:
$$\frac{dC_A}{dt} = -k_1(T) C_A$$

### Component B:
$$\frac{dC_B}{dt} = k_1(T) C_A - k_2(T) C_B$$

### Component C:
$$\frac{dC_C}{dt} = k_2(T) C_B$$

**Where:**
- $C_A, C_B, C_C$ = concentrations of components A, B, and C
- $t$ = time

## Energy Balance

The energy balance for the reacting mass is:

$$\rho C_p \frac{dT}{dt} = -\Delta H_1 k_1(T) C_A - \Delta H_2 k_2(T) C_B + \frac{UA}{V}(T_j - T)$$

**Where:**
- $\rho$ = density
- $C_p$ = heat capacity
- $\Delta H_1, \Delta H_2$ = heats of reaction
- $U$ = overall heat transfer coefficient
- $A$ = heat transfer area
- $V$ = reactor volume
- $T_j$ = jacket temperature
- $T$ = reactor temperature

## Notes on Extension

This model can be extended to include:
- Explicit jacket energy balance
- Temperature controller dynamics
- Measurement noise and disturbances
- Hybrid data-driven correction terms

These extensions are consistent with digital twin and advanced process
modeling workflows.



In [ ]:
# Import libraries
from __future__ import annotations
from dataclasses import dataclass
from typing import Callable, Dict, Tuple
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Load data
data = pd.read_csv('../data/simulated_batch_data.csv')
data.head()

# Implement mechanistic model

The mechanistic model (physics) provides the baseline prediction. 

The mechanistic model is saved in this location:
___/Users/paula/Documents/Process_Engineering_Simulations_MachineLearning/hybrid-digital-twin-pharma-style/model/mechanistic_model.py___

Assuming the values of our parameters, and then finally plotting it against the simulated batch data.

In [ ]:
# Import models
import sys
sys.path.append('..')

from model.mechanistic_model import MechanisticModel, Params, default_Tj_profile

params = Params(
    k01=8.0,
    E1=10000,
    k02=2.0,
    E2=15000,
    dH1=-45000,
    dH2=-35000,
    rhoCp=4200,
    UA_over_V=200
)

Tj = default_Tj_profile(330.0)

model = MechanisticModel(params, Tj)

CA0 = data["concentration"].iloc[0]
CB0 = 0.0
CC0 = 0.0
T0  = data["temperature"].iloc[0] + 273.15  # convert °C → K if needed

y0 = np.array([CA0, CB0, CC0, T0])

t_eval = data["time"].values

results = model.simulate(
    (t_eval.min(), t_eval.max()),
    y0,
    t_eval=t_eval
)

import matplotlib.pyplot as plt

plt.figure(figsize=(10,4))
plt.plot(results["t"], results["CA"], label="Model CA")
plt.scatter(data["time"], data["concentration"], color="red", label="Measured")
plt.xlabel("Time")
plt.ylabel("Concentration")
plt.legend()
plt.title("Mechanistic Model vs Measured Data")
plt.show()

## Train hybrid correction layer

In this layer, Machine Learning allows us to "learn" the "personality" of the manufacturing plant by correcting our mechanistic model's baseline prediction.

For this digital twin model we will pick something to correct from either of the following:

1. Correct the measured output directly (easy, common)
  $$ y_{true}​(t)≈y_{mech}​(t)+Δy_{ML}​(x(t))$$
2. Correct the dynamics (more advanced and the ML adds terms to the ODEs:
  $$\dot{y} = f_{mech}​(t,y,u)+g_{ML}​(t,y,u)$$

For simplicity we'll start with 1 and extend to 2 as we progress.

Looking back at our simulated batch data we have the following columns:
* time
* temperature
* pressure
* concentration (A, our reactant)

Therefore we can train using ML to predict the residual such that:
 $$r(t)=C_{A,measured}​(t)−C_{A,mech}​(t)$$

Then the corrected prediction is:
 $$C_{A,hybrid}​(t) = C_{A,mech}​(t) + \hat{r}​(t)$$ 

We will start with something foregiving and non-fussy:
* random forest regression model
* gradient boosting

## Random Forest
A Random Forest is an ensemble of decision trees.

Each tree partitions the feature space into regions and predicts a constant in each region.

A single decision tree approximates:
$$
r(x) =
\begin{cases}
c_1 & \text{if } x \in R_1 \\
c_2 & \text{if } x \in R_2 \\
\vdots
\end{cases}
$$

Where $$R_{i}$$ are regions created by splits like:
$$T<35°c?$$
$$time<5?$$
$$𝐶_{A,mech}>0.7?$$

A Random Forest averages many such trees:
$$
r(x) = \frac{1}{M}\sum_{m=1}^{M} T_m(x)
$$
where $T_m$ are individual trees, giving a smooth non-linear approximation.

## Model Evaluation
We evaluate our model using the following:

* MSE measures average squared error.
$$
\mathrm{MSE} = \frac{1}{n} \sum_{i=1}^{n} \left( y_i - \hat{y}_i \right)^2
$$

* RMSE brings it back to the original units.
$$
\mathrm{RMSE} = \sqrt{ \frac{1}{n} \sum_{i=1}^{n} \left( y_i - \hat{y}_i \right)^2 }
$$

* R² compares your model’s error to the error you’d get by just predicting the mean every time.
$$
R^2 = 1 - 
\frac{
\sum_{i=1}^{n} \left( y_i - \hat{y}_i \right)^2
}{
\sum_{i=1}^{n} \left( y_i - \bar{y} \right)^2
}
$$

   The denominator in R² is the total variance.

In [ ]:
# This uses scikit-learn.
import sys
!{sys.executable} -m pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd

# Data Splitting
from sklearn.model_selection import train_test_split
# Random Forest Regression
from sklearn.ensemble import RandomForestRegressor
# Metrics to evaluate prediction quality
from sklearn.metrics import mean_squared_error, r2_score

# 1) Copy Dataset. This prevents us from modifying original data
df = data.copy()

# 2) Add Mechanistic prediction sampled at same times, aligning mechanistic model with measured timestamps
# Mathematically we now have C_{a,mech}​(t) and the C_{a,measured}​(t)  
df["CA_mech"] = results["CA"]  # results computed using t_eval = data["time"].values

# 3) Define Residual target (target for ML): what physics missed
df["residual_CA"] = df["concentration"] - df["CA_mech"]

# 4) Define the Features: what we allow ML to look at. Given these inputs, we ask ML to predict the residual r.
X = df[["time", "temperature", "pressure", "CA_mech"]].values
y = df["residual_CA"].values

# 5) Split data into train/test. We do have a tiny dataset, but for now it demonstrates workflow. We will test on 30%
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0
)

# 6) Train ML correction (Random Forest)
corr = RandomForestRegressor(n_estimators=300, random_state=0)
corr.fit(X_train, y_train)

# 7) Evaluate
y_pred = corr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)
print("R2:", r2_score(y_test, y_pred))

# 5) Build hybrid prediction for all points
df["residual_hat"] = corr.predict(X)
df["CA_hybrid"] = df["CA_mech"] + df["residual_hat"]

# 6) Plot it
plt.figure(figsize=(10,4))
plt.plot(df["time"], df["CA_mech"], label="Mechanistic CA")
plt.plot(df["time"], df["CA_hybrid"], label="Hybrid CA")
plt.scatter(df["time"], df["concentration"], label="Measured", marker="o")
plt.xlabel("Time")
plt.ylabel("Concentration")
plt.legend()
plt.title("Mechanistic vs Hybrid vs Measured")
plt.show()

# Interpretation / Discussion

1. Our $RMSE$ values tell us that our model is off by 0.05 concentration units of A, about 5.4% in absolute error. In my opinion, it's not that bad.
2. Our $R^2$ is relatively low. Our model only explains about 73.57% of the variability in the test data (but we do only have 10 data points, trained on 7, tested on 3).

This validates the hybrid digital twin architecture concept that may help pharmaceutical plants optimize their production operating conditions, and predict yield when kinetics and process dynamics are too complex to model from first principles. 

A perfect mechanistic model that captures parallel side reactions, impurity formation pathways, catalyst aging, heat transfer non-linearities, mixing limitations, polymorph transformations, among others is just impossible. Hybrid modeling brings reality to the pure physics that mechanistic models bring.

# Next Steps
1. Use the twin to maximize the production of B.
2. Imolement a temperature sweep optimizer
3. Plot B_max vs jacket temperature